# SlideRule Voila Demo

#### SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

This web page is _not_ SlideRule but is a demonstration of using public SlideRule APIs to return elevations within a small region of interest.  The web page is implemented inside a Jupyter Notebook using SlideRule's Python client, and is statically served using Voila.  For more information on SlideRule, and how to install and use it for your own analysis applications, please see our website at [slideruleearth.io](https://slideruleearth.io).

For more detailed example notebooks that use SlideRule APIs, check out these example notebooks at [github.com/SlideRuleEarth/sliderule-python](https://github.com/SlideRuleEarth/sliderule-python/tree/main/examples).

### Surface Elevations ([atl06p](https://slideruleearth.io/rtd/api_reference/icesat2.html#atl06p))

In [ ]:
import warnings
warnings.filterwarnings('ignore') # turn off warnings for demo

In [ ]:
# load the necessary packages
from io import BytesIO
from sliderule import icesat2, ipysliderule, io, sliderule
import ipywidgets as widgets
from concurrent.futures import ThreadPoolExecutor
import geopandas
import logging
import base64
import time
import copy
import json
import re
from IPython import display
# atl03 plotting imports
import numpy as np
import matplotlib.pyplot as plt
# autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# create global variables
atl06_rsps = None
atl06_parms = None
SRwidgets = ipysliderule.widgets()
points_dropdown = None
update_button = widgets.Button(description="Update Map")
run_button = widgets.Button(description="Run SlideRule!")
run_output = widgets.Output()
download_output = widgets.Output()
download_atl06_button = widgets.Button(description="Download File")
download_atl03_button = widgets.Button(description="Download File")
SRwidgets.file_format.options = ["GeoJSON","csv","geoparquet"]
SRwidgets.file_format.value = 'geoparquet'
show_code06_button = widgets.Button(description="Show Code")
show_code06_output = widgets.Output()

In [ ]:
def create_map(projection):
    # create ipyleaflet map in specified projection
    global m
    m = ipysliderule.leaflet(projection)
    # install click handler callback
    m.add_selected_callback(SRwidgets.atl06_click_handler)
    display.display(m.map)

# interactively change map when projection widget is changed
out = widgets.interactive_output(create_map, dict(projection=SRwidgets.projection))
display.display(out)
display.display(run_output)

In [ ]:
# update map
def on_update_clicked(b):
    m.add_layer(
        layers=SRwidgets.layers.value,
        rendering_rule=SRwidgets.rendering_rule
    )

# map widgets
display.display(widgets.VBox([
    SRwidgets.projection,
    SRwidgets.layers,
    SRwidgets.raster_functions
]))

# update button
update_button.on_click(on_update_clicked)
display.display(update_button)

In [ ]:
%matplotlib inline
granule_count = 0

# callbacks for events and exceptions
def demo_logeventrec(rec):
    # print(f'{rec["attr"]}                                                        \r', end="")
    pass

def demo_exceptrec(rec):
    global granule_count
    if "Successfully" in rec["text"]:
        tokens = rec["text"].split()
        tokens[4] = f'[{granule_count}'
        text = ' '.join(tokens)
        print(f'{text}                                                        \r', end="")
        granule_count += 1

# build and transmit requests to SlideRule
def runSlideRule():
    global atl06_parms, granule_count

    # reset granule count
    granule_count = 0

    # set the url for the sliderule service
    icesat2.init("slideruleearth.io", loglevel=logging.ERROR, max_resources=1000)

    # build sliderule parameters using latest values from widget
    atl06_parms = {
        # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
        "srt": icesat2.SRT_DYNAMIC,
        # length of ATL06-SR segment in meters
        "len": SRwidgets.length.value,
        # step distance for successive ATL06-SR segments in meters
        "res": SRwidgets.step.value,
        # confidence level for PE selection
        "cnf": SRwidgets.confidence.value,
        # ATL08 land surface classifications
        "atl08_class": list(SRwidgets.land_class.value),
        # maximum iterations, not including initial least-squares-fit selection
        "maxi": SRwidgets.iteration.value,
        # minimum along track spread
        "ats": SRwidgets.spread.value,
        # minimum PE count
        "cnt": SRwidgets.count.value,
        # minimum height of PE window in meters
        "H_min_win": SRwidgets.window.value,
        # maximum robust dispersion in meters
        "sigma_r_max": SRwidgets.sigma.value
    }

    # clear existing geodataframe results
    elevations = [sliderule.emptyframe()]
    sliderule.logger.warning('No valid regions to run') if not m.regions else None

    # for each region of interest
    for poly in m.regions:
        # add polygon from map to sliderule parameters
        atl06_parms["poly"] = poly
        # make the request to the SlideRule (ATL06-SR) endpoint
        # and pass it the request parameters to request ATL06 Data
        elevations.append(icesat2.atl06p(atl06_parms, callbacks={'eventrec': demo_logeventrec, 'exceptrec': demo_exceptrec}))

    # return concatenated set of results
    gdf = geopandas.pd.concat(elevations)
    return gdf

# run sliderule action
def on_run_clicked(b):
    global atl06_rsps, points_dropdown, stride
    with run_output:
        print(f'SlideRule processing request... initiated\r', end="")
        perf_start = time.perf_counter()
        atl06_rsps = runSlideRule()
        perf_duration = time.perf_counter() - perf_start
        print(f'SlideRule processing request... completed in {perf_duration:.3f} seconds; returned {atl06_rsps.shape[0]} elevations                                   ')
        if atl06_rsps.shape[0] > 0:
            max_plot_points = 10000
            if points_dropdown.value == "100K":
                max_plot_points = 100000
            elif points_dropdown.value == "all":
                max_plot_points = np.inf
            # limit number of points to plot
            plot_points = np.minimum(max_plot_points, atl06_rsps.shape[0])
            stride = int(atl06_rsps.shape[0]//plot_points)
            print(f'Plotting {plot_points} of {atl06_rsps.shape[0]} elevations. This may take 10-60+ seconds for larger point datasets.')
            fields = atl06_rsps.leaflet.default_atl06_fields()
            atl06_rsps.leaflet.GeoData(m.map,
                column_name=SRwidgets.variable.value,
                cmap=SRwidgets.colormap,
                stride=stride,
                tooltip=True,
                colorbar=True,
                fields=fields
            )
            # install handlers and callbacks
            atl06_rsps.leaflet.set_observables(SRwidgets)
            atl06_rsps.leaflet.add_selected_callback(SRwidgets.atl06_click_handler)
            m.add_region_callback(atl06_rsps.leaflet.handle_region)

# refresh action
def refresh_leaflet_draw(sender):
    global atl06_rsps, points_dropdown, stride
    if atl06_rsps is not None and atl06_rsps.shape[0] > 0:
        if isinstance(sender['new'], str) and (sender['new'] == "10K"):
            max_plot_points = 10000
        elif isinstance(sender['new'], str) and (sender['new'] == "100K"):
            max_plot_points = 100000
        elif isinstance(sender['new'], str) and (sender['new'] == "all"):
            max_plot_points = np.inf
        else:
            return
        # limit number of points to plot
        plot_points = np.minimum(max_plot_points, atl06_rsps.shape[0])
        stride_new = int(atl06_rsps.shape[0]//plot_points)
        if (stride_new == stride):
            return
        # update stride
        stride = stride_new
        print(f'Plotting {plot_points} of {atl06_rsps.shape[0]} elevations. This may take 10-60+ seconds for larger point datasets.')
        # sliced geodataframe for plotting
        atl06_rsps.leaflet._gdf_selected = atl06_rsps.leaflet._gdf[slice(None,None,stride)]
        atl06_rsps.leaflet._gdf_selected['data'] = atl06_rsps.leaflet._gdf_selected[atl06_rsps.leaflet.column_name]
        atl06_rsps.leaflet.redraw()

# show code action
global code06_is_shown
code06_is_shown = False
def on_show_code06_clicked(b):
    global url_textbox, atl06_parms, code06_is_shown
    with show_code06_output:
        display.clear_output()
        code06_is_shown = not code06_is_shown
        if code06_is_shown:
            print(f'icesat2.init()')
            # validate boolean entries to be in title case
            atl06_json = json.dumps(atl06_parms, indent=4)
            atl06_json = re.sub(r'\b(true|false)', lambda m: m.group(1).title(), atl06_json)
            print('parms = ', atl06_json, sep='')
            print('gdf = icesat2.atl06p(parms)')
            show_code06_button.description = "Hide Code"
        else:
            show_code06_button.description = "Show Code"


# Download ATL06-SR data as geojson
display.display(download_output)
def download_file(gdf, filename, mime_type='text/json'):
    if (mime_type == 'text/json'):
        content = base64.b64encode(gdf.to_json().encode()).decode()
    elif (mime_type == 'text/csv'):
        df1 = gdf.reset_index().rename(columns={"index":"time"}).drop(columns=['geometry'])
        df2 = geopandas.pd.DataFrame(
            np.c_[gdf.geometry.x.values, gdf.geometry.y.values],
            columns=['longitude','latitude']
        )
        df = geopandas.pd.concat([df1, df2], axis=1)
        content = base64.b64encode(df.to_csv(index=False).encode()).decode()
    elif (mime_type == 'application/vnd.apache.parquet'):
        fid = BytesIO()
        parms = copy.copy(atl06_parms)
        version = sliderule.get_version()
        parms['version'] = version['icesat2']['version']
        parms['commit'] = version['icesat2']['commit']
        io.to_parquet(gdf, fid, parameters=parms, regions=m.regions)
        content = base64.b64encode(fid.getbuffer()).decode()
    # create download link
    url = f'data:{mime_type};charset=utf-8;base64,{content}'
    js = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{url}');
        a.click();
    """
    with download_output:
        display.clear_output()
        display.display(display.HTML(f'<script>{js}</script>'))

def on_atl06_download_clicked(e=None):
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(download_file,
            atl06_rsps, SRwidgets.atl06_filename,
            mime_type=SRwidgets.mime_type)
        future.result()

# link buttons
run_button.on_click(on_run_clicked)
show_code06_button.on_click(on_show_code06_clicked)
download_atl06_button.on_click(on_atl06_download_clicked)

In [ ]:
# points to plot drop down
points_dropdown = widgets.Dropdown(
    options = ["10K", "100K", "all"],
    value = "10K",
    description = "Pts to Draw",
    disabled = False,
)
points_dropdown.observe(refresh_leaflet_draw)

# display widgets for setting SlideRule parameters
display.display(widgets.VBox([
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
    SRwidgets.variable,
    SRwidgets.cmap,
    points_dropdown,
    SRwidgets.reverse,
]))

# display buttons
display.display(run_button)
display.display(SRwidgets.HBox([download_atl06_button, SRwidgets.file_format]))
display.display(show_code06_button, show_code06_output)

In [ ]:
# globals for atl03 processing
atl03_rsps = None
atl03_parms = None
show_code03_button = widgets.Button(description="Show Code")
show_code03_output = widgets.Output()
elev_dropdown = None
pc_button = widgets.Button(description="Plot Photon Cloud")
pc_output = widgets.Output()

# track lookup tables
gt2str = {icesat2.GT1L: 'gt1l', icesat2.GT1R: 'gt1r', icesat2.GT2L: 'gt2l', icesat2.GT2R: 'gt2r', icesat2.GT3L: 'gt3l', icesat2.GT3R: 'gt3r'}
str2gt = {'gt1l': icesat2.GT1L, 'gt1r': icesat2.GT1R, 'gt2l': icesat2.GT2L, 'gt2r': icesat2.GT2R, 'gt3l': icesat2.GT3L, 'gt3r': icesat2.GT3R}
gtlookup = {icesat2.GT1L: 1, icesat2.GT1R: 1, icesat2.GT2L: 2, icesat2.GT2R: 2, icesat2.GT3L: 3, icesat2.GT3R: 3}
pairlookup = {icesat2.GT1L: 0, icesat2.GT1R: 1, icesat2.GT2L: 0, icesat2.GT2R: 1, icesat2.GT3L: 0, icesat2.GT3R: 1}

### Photon Cloud ([atl03sp](https://slideruleearth.io/rtd/api_reference/icesat2.html#atl03sp))

In [ ]:
%matplotlib widget
# ATL03 Subsetter
def runATL03Subsetter():
    global atl03_parms

    icesat2.init("slideruleearth.io", loglevel=logging.ERROR)

    # build sliderule parameters using latest values from widget
    atl03_parms = {
        # processing parameters
        "srt": icesat2.SRT_DYNAMIC,
        "len": SRwidgets.length.value,
        "res": SRwidgets.step.value,

        # classification and checks
        "pass_invalid": True, # still return photon segments that fail checks
        "cnf": -2, # all photons
        "atl08_class": ["atl08_noise", "atl08_ground", "atl08_canopy", "atl08_top_of_canopy", "atl08_unclassified"],
        "yapc": {"score": 0}, # all photons
        "ats": SRwidgets.spread.value,
        "cnt": SRwidgets.count.value,

        # region of interest
        "poly": m.regions[0],

        # track selection
        "rgt": int(SRwidgets.rgt.value),
        "cycle": int(SRwidgets.cycle.value),
        "track": gtlookup[str2gt[SRwidgets.ground_track.value]]
    }

    # make call to sliderule
    rsps = icesat2.atl03sp(atl03_parms)

    # return geodataframe
    return rsps

# photon_cloud action
def on_pc_clicked(b):
    global atl03_rsps, atl06_rsps, elev_dropdown, fig
    with pc_output:
        pc_output.clear_output(True)

        # Run ATL03 Subsetter
        print(f'SlideRule processing request... initiated\r', end="")
        perf_start = time.perf_counter()
        atl03_rsps = runATL03Subsetter()
        perf_duration = time.perf_counter() - perf_start
        print(f'SlideRule processing request... completed in {perf_duration:.3f} seconds; returned {atl03_rsps.shape[0]} records                                   ')

        # Create Plots
        if atl03_rsps.shape[0] > 0 and atl06_rsps.shape[0] > 0:
            fig,ax = plt.subplots(num=None, figsize=(10, 8))
            fig.set_facecolor('white')
            fig.canvas.header_visible = False
            ax.set_title("Photon Cloud")
            ax.set_ylabel('height (m)')
            # start at the first segment
            x_offset = atl03_rsps['segment_dist'].min()
            # plot ATL03 and ATL06 data
            atl03_rsps.icesat2.plot(ax=ax, kind='scatter',
                data_type='atl03', cmap=SRwidgets.colormap,
                classification=SRwidgets.plot_classification.value,
                x_offset=x_offset, legend=True, legend_frameon=True,
                **SRwidgets.plot_kwargs)
            if (elev_dropdown.value == 'enabled'):
                atl06_rsps.icesat2.plot(ax=ax, kind='scatter',
                    data_type='atl06', x_offset=x_offset,
                    legend=True, legend_frameon=True,
                    **SRwidgets.plot_kwargs)
            # draw and show plot
            plt.show()
            fig.canvas.draw()

def refresh_ATL03_plot(sender):
    global atl03_rsps, atl06_rsps, elev_dropdown, fig
    if isinstance(sender['new'], str):
        # reset figure and axes
        fig.clear()
        fig.set_facecolor('white')
        fig.canvas.header_visible = False
        ax = fig.add_subplot(111)
        # reset title and labels
        ax.set_title("Photon Cloud")
        ax.set_ylabel('height (m)')
        # start at the first segment
        x_offset = atl03_rsps['segment_dist'].min()
        # plot ATL03 and ATL06 data
        atl03_rsps.icesat2.plot(ax=ax, kind='scatter',
            data_type='atl03', cmap=SRwidgets.colormap,
            classification=SRwidgets.plot_classification.value,
            x_offset=x_offset, legend=True, legend_frameon=True,
            **SRwidgets.plot_kwargs)
        if (elev_dropdown.value == 'enabled'):
            atl06_rsps.icesat2.plot(ax=ax, kind='scatter',
                data_type='atl06', x_offset=x_offset,
                legend=True, legend_frameon=True,
                **SRwidgets.plot_kwargs)
        # draw and show plot
        fig.canvas.draw()
        
# create button to display geodataframe
pc_button.on_click(on_pc_clicked)

# click handler for individual photons
def click_handler(feature):
    if "properties" in feature:
        if "rgt" in feature["properties"] and "cycle" in feature["properties"] and "gt" in feature["properties"]:
            SRwidgets.rgt.value = str(feature["properties"]["rgt"])
            SRwidgets.cycle.value = str(feature["properties"]["cycle"])
            SRwidgets.ground_track.value = gt2str[feature["properties"]["gt"]]

# install click handler callback
m.add_selected_callback(click_handler)

# show code action
global code03_is_shown
code03_is_shown = False
def on_show_code03_clicked(b):
    global url_textbox, atl03_parms, code03_is_shown
    with show_code03_output:
        display.clear_output()
        code03_is_shown = not code03_is_shown
        if code03_is_shown:
            print(f'icesat2.init()')
            # validate boolean entries to be in title case
            atl03_json = json.dumps(atl03_parms, indent=4)
            atl03_json = re.sub(r'\b(true|false)', lambda m: m.group(1).title(), atl03_json)
            print('parms = ', atl03_json, sep='')
            print('gdf = icesat2.atl03sp(parms)')
            show_code03_button.description = "Hide Code"
        else:
            show_code03_button.description = "Show Code"

def on_atl03_download_clicked(e=None):
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(download_file,
            atl03_rsps, SRwidgets.atl03_filename,
            mime_type=SRwidgets.mime_type)
        future.result()

# install click handler callback
show_code03_button.on_click(on_show_code03_clicked)
download_atl03_button.on_click(on_atl03_download_clicked)

In [ ]:
# elevation plot drop down
elev_dropdown = widgets.Dropdown(
    options = ["enabled", "disabled"],
    value = "enabled",
    description = "ATL06-SR",
    disabled = False,
)

display.display(SRwidgets.rgt)
display.display(SRwidgets.cycle)
display.display(SRwidgets.ground_track)
display.display(SRwidgets.plot_classification)
display.display(elev_dropdown)
display.display(pc_button)
display.display(pc_output)
display.display(SRwidgets.HBox([download_atl03_button, SRwidgets.file_format]))
display.display(show_code03_button, show_code03_output)
SRwidgets.plot_classification.observe(refresh_ATL03_plot)
elev_dropdown.observe(refresh_ATL03_plot)